In [1]:
%cd ..

C:\Users\LiuJinbao\Documents\Code\PlasmaChemistry_Copy


In [10]:
from plasmistry.molecule import get_vib_energy
from plasmistry import constants as const

In [3]:
get_vib_energy("CO2",quantum_number=(0,0,1),minimum_is_zero=True)

0.2912574773137701

In [7]:
vib_energy_seq = np.array([get_vib_energy("CO2",quantum_number=(0,0,_),minimum_is_zero=True) for _ in range(22)])

In [11]:
Tvib=4000
np.exp(-vib_energy_seq*const.eV2K/Tvib)

array([1.00000000e+00, 4.29567687e-01, 1.86191208e-01, 8.14296733e-02,
       3.59337196e-02, 1.59999128e-02, 7.18834669e-03, 3.25863992e-03,
       1.49052646e-03, 6.87921633e-04, 3.20356988e-04, 1.50530809e-04,
       7.13694827e-05, 3.41425269e-05, 1.64806650e-05, 8.02693526e-06,
       3.94476148e-06, 1.95608488e-06, 9.78702284e-07, 4.94093871e-07,
       2.51689032e-07, 1.29364488e-07])

In [23]:
ratio_seq = np.exp(-vib_energy_seq*const.eV2K/Tvib)
total_density = 1.2e18
for _ in range(22):
    print(f"CO2(v{_:.0f})", end=" ")
    print(total_density/ratio_seq.sum()*ratio_seq[_])

CO2(v0) 6.809065804417496e+17
CO2(v1) 2.924954647973315e+17
CO2(v2) 1.267788187131161e+17
CO2(v3) 5.544600041487556e+16
CO2(v4) 2.446750612949822e+16
CO2(v5) 1.0894445899353524e+16
CO2(v6) 4894592565425459.0
CO2(v7) 2218829364000352.5
CO2(v8) 1014909276473236.6
CO2(v9) 468410366834991.2
CO2(v10) 218133181364772.0
CO2(v11) 102497418486921.62
CO2(v12) 48595950403022.336
CO2(v13) 23247871252779.008
CO2(v14) 11221793233423.432
CO2(v15) 5465593041788.417
CO2(v16) 2686014049989.3237
CO2(v17) 1331911065893.1233
CO2(v18) 666404825489.197
CO2(v19) 336431768119.2252
CO2(v20) 171376718101.81656
CO2(v21) 88085130954.55788


In [21]:
ratio_seq.sum()

1.762356297425529